In [9]:
import pandas as pd
import numpy as np
import os
import joblib
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from collections import Counter

# Definir las variables de entrada
model_folder = '4146_PF4_A218izq_T12'
model_name = 'ensemble_100'
window_size = int(model_name.split('_')[-1])

# Cargar el modelo y el scaler
model_path = os.path.join('modelos', model_folder, f'{model_name}.pkl')
scaler_path = os.path.join('modelos', model_folder, f'scaler_{window_size}.pkl')

ensemble_model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

# Función para preparar los datos
def prepare_data(df, window_size):
    X = []
    y = []
    for i in range(0, df.shape[1], 2):
        if i + 1 >= df.shape[1]:
            break
        angulo = df.iloc[1:, i].dropna().astype(float).values
        par = df.iloc[1:, i+1].dropna().astype(float).values
        label = df.iloc[0, i]
        min_len = min(len(angulo), len(par))
        if min_len > window_size:
            for j in range(0, min_len - window_size + 1, window_size):
                X.append([angulo[j:j + window_size], par[j:j + window_size]])
                y.append(label)
    X = np.array(X).reshape(-1, 2 * window_size)  # Aplanar las ventanas
    y = np.array(y)
    y = np.where(y == 2, 1, y)  # Convertir etiquetas 2 a 1
    return X, y

# Función para procesar un archivo de Excel
def process_excel(file_path, window_size, model, scaler):
    # Cargar y preparar los datos
    df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)
    df.columns = [f'Col_{i}' for i in range(len(df.columns))]
    print(f"Procesando archivo: {file_path}")
    print("Forma inicial del DataFrame:", df.shape)
    print("Primeras 5 filas del DataFrame original:")
    print(df.head())
    
    # Eliminar la primera y tercera columna
    df = df.drop(df.columns[[0, 2]], axis=1)
    df = df.dropna(axis=1, how='all').dropna(how='all')
    print("\nForma del DataFrame después de eliminar filas y columnas vacías:", df.shape)
    
    df_transposed = df.transpose().reset_index(drop=True)
    df_transposed.columns = [f'Col_{i}' for i in range(len(df_transposed.columns))]
    print("\nPrimeras 5 filas del DataFrame transpuesto:")
    print(df_transposed.head())
    
    # Convertir a numérico y limpiar
    for col in df_transposed.columns:
        df_transposed[col] = pd.to_numeric(df_transposed[col], errors='coerce')
    df_transposed = df_transposed.dropna(how='all')
    print("Valores únicos en la primera fila:", df_transposed.iloc[0].unique())
    
    # Preparar los datos
    X, y = prepare_data(df_transposed, window_size)
    if len(X) == 0:
        print("No hay suficientes datos para procesar este archivo.")
        return None, None

    # Escalar los datos
    X_scaled = scaler.transform(X)
    
    # Realizar predicciones
    y_pred_probs = model.predict_proba(X_scaled)[:, 1]
    y_pred = model.predict(X_scaled)
    
    # Calcular métricas
    roc_auc = roc_auc_score(y, y_pred_probs)
    print("\nModel Performance:")
    print(classification_report(y, y_pred))
    print("ROC AUC:", roc_auc)
    
    # Guardar métricas en un archivo CSV
    metrics_df = pd.DataFrame({
        'Modelos': [f'{model_folder}/{model_name}'],
        'ROC AUC': [roc_auc]
    })
    metrics_csv_path = 'resultados/metricas_predicciones.csv'
    if os.path.exists(metrics_csv_path):
        metrics_df.to_csv(metrics_csv_path, mode='a', header=False, index=False)
    else:
        metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"\nMétricas guardadas en: {metrics_csv_path}")
    
    return y, y_pred

# Ejemplo de uso con un archivo de Excel
excel_file_path = r'src\data\4146_PF4_A218izq_T12\4_a.xlsx'
y_true, y_pred = process_excel(excel_file_path, window_size, ensemble_model, scaler)


Procesando archivo: src\data\4146_PF4_A218izq_T12\4_a.xlsx
Forma inicial del DataFrame: (1500, 754)
Primeras 5 filas del DataFrame original:
                    Col_0  Col_1                 Col_2     Col_3     Col_4  \
0 2024-06-12 06:00:57.217      0  4146 PF4 A218izq T12  0.000000  0.000000   
1 2024-06-12 06:00:57.217      0  4146 PF4 A218izq T12  0.043945  0.055664   
2 2024-06-12 06:02:35.167      0  4146 PF4 A218izq T12  0.000000 -0.206308   
3 2024-06-12 06:02:35.167      0  4146 PF4 A218izq T12  0.020508  0.026367   
4 2024-06-12 06:04:17.813      0  4146 PF4 A218izq T12  0.000000  0.000000   

      Col_5     Col_6     Col_7     Col_8     Col_9  ...  Col_744  Col_745  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  ...      NaN      NaN   
1  0.052734  0.055664  0.046875  0.058594  0.067383  ...      NaN      NaN   
2  0.000000 -0.206308  0.000000  0.000000  0.000000  ...      NaN      NaN   
3  0.023438  0.029297  0.038086  0.026367  0.014648  ...      NaN      NaN   


In [15]:
import pandas as pd
import numpy as np
import os
import joblib
from glob import glob

# Definir las variables de entrada
model_folder = '4146_PF4_A218izq_T12'
model_name = 'ensemble_100'
window_size = int(model_name.split('_')[-1])

# Cargar el modelo y el scaler
model_path = os.path.join('modelos', model_folder, f'{model_name}.pkl')
scaler_path = os.path.join('modelos', model_folder, f'scaler_{window_size}.pkl')

ensemble_model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

# Función para preparar los datos
def prepare_data_for_prediction(df, window_size):
    X = []
    for i in range(0, df.shape[1], 2):
        if i + 1 >= df.shape[1]:
            break
        angulo = df.iloc[:, i].dropna().astype(float).values
        par = df.iloc[:, i + 1].dropna().astype(float).values
        min_len = min(len(angulo), len(par))
        if min_len > window_size:
            for j in range(0, min_len - window_size + 1, window_size):
                X.append([angulo[j:j + window_size], par[j:j + window_size]])
    X = np.array(X).reshape(-1, 2 * window_size)  # Aplanar las ventanas
    return X

# Función para procesar un archivo de Excel y hacer predicciones
def process_excel_for_prediction(file_path, window_size, model, scaler):
    # Cargar y preparar los datos
    df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)
    df.columns = [f'Col_{i}' for i in range(len(df.columns))]
    print(f"Procesando archivo: {file_path}")
    print("Forma inicial del DataFrame:", df.shape)
    print("Primeras 5 filas del DataFrame original:")
    print(df.head())
    
    # Eliminar la primera y tercera columna
    df = df.drop(df.columns[[0, 2]], axis=1)
    df = df.dropna(axis=1, how='all').dropna(how='all')
    print("\nForma del DataFrame después de eliminar filas y columnas vacías:", df.shape)
    
    df_transposed = df.transpose().reset_index(drop=True)
    df_transposed.columns = [f'Col_{i}' for i in range(len(df_transposed.columns))]
    print("\nPrimeras 5 filas del DataFrame transpuesto:")
    print(df_transposed.head())
    
    # Convertir a numérico y limpiar
    for col in df_transposed.columns:
        df_transposed[col] = pd.to_numeric(df_transposed[col], errors='coerce')
    df_transposed = df_transposed.dropna(how='all')
    print("Valores únicos en la primera fila:", df_transposed.iloc[0].unique())
    
    # Preparar los datos
    X = prepare_data_for_prediction(df_transposed, window_size)
    if len(X) == 0:
        print("No hay suficientes datos para procesar este archivo.")
        return

    # Escalar los datos
    X_scaled = scaler.transform(X)
    
    # Realizar predicciones en tiempo real
    for i, window in enumerate(X_scaled):
        window = window.reshape(1, -1)  # Ajustar la forma de la ventana
        pred = model.predict(window)
        result = 'OK' if pred[0] == 0 else 'NOT OK'
        print(f"Predicción para la ventana {i + 1}: {result}")

# Ejemplo de uso con un archivo de Excel
excel_file_path = r'src\data\4146_PF4_A218izq_T12\Prueba.xlsx'
process_excel_for_prediction(excel_file_path, window_size, ensemble_model, scaler)


Procesando archivo: src\data\4146_PF4_A218izq_T12\Prueba.xlsx
Forma inicial del DataFrame: (6, 741)
Primeras 5 filas del DataFrame original:
                    Col_0  Col_1                 Col_2     Col_3     Col_4  \
0 2024-06-12 06:00:57.217      0  4146 PF4 A218izq T12  0.000000  0.000000   
1 2024-06-12 06:00:57.217      0  4146 PF4 A218izq T12  0.043945  0.055664   
2 2024-06-12 06:02:35.167      0  4146 PF4 A218izq T12  0.000000 -0.206308   
3 2024-06-12 06:02:35.167      0  4146 PF4 A218izq T12  0.020508  0.026367   
4 2024-06-12 06:27:02.560      2  4146 PF4 A218izq T12  0.000000 -0.103154   

      Col_5     Col_6     Col_7     Col_8     Col_9  ...      Col_731  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  ...          NaN   
1  0.052734  0.055664  0.046875  0.058594  0.067383  ...          NaN   
2  0.000000 -0.206308  0.000000  0.000000  0.000000  ...          NaN   
3  0.023438  0.029297  0.038086  0.026367  0.014648  ...          NaN   
4 -0.103154 -0.103154  0.